# MLB Player Performance Correlation Analysis

### By David Montoto

## Abstract: 
This analysis aims to investigate the correlation between individual player performance, measured by Wins Above Replacement (WAR), and team success metrics in Major League Baseball (MLB). Using a comprehensive dataset that includes player WAR, team win-loss records, playoff appearances, and championships, we calculate correlation coefficients to quantify the relationship between these variables. By visualizing the data through scatter plots with regression lines and performing statistical tests, we assess the significance of the observed correlations. This study provides insights into the impact of star players on team performance, offering valuable information for team management and strategic decision-making in professional baseball.

## Goal:
1. Analyze Player Impact: Examine how the performance of top players influences overall team success, employing regression and feature importance analysis

### Data Cleaning and Preprocessing

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
df = pd.read_csv('baseballdata.csv')
df

,Unnamed: 0,Rk,Year,Tm,Lg,G,W,L,Ties,W.L.,...,R,RA,Attendance,BatAge,PAge,X.Bat,X.P,Top.Player,Managers,current
0,1,1,2016,Arizona Diamondbacks,NL West,162,69,93,0,0.426,...,752,890,"2,036,216",26.7,26.4,50,29,J.Segura (5.7),C.Hale (69-93),Arizona Diamondbacks
1,2,2,2015,Arizona Diamondbacks,NL West,162,79,83,0,0.488,...,720,713,"2,080,145",26.6,27.1,50,27,P.Goldschmidt (8.8),C.Hale (79-83),Arizona Diamondbacks
2,3,3,2014,Arizona Diamondbacks,NL West,162,64,98,0,0.395,...,615,742,"2,073,730",27.6,28.0,52,25,P.Goldschmidt (4.5),K.Gibson (63-96) and A.Trammell (1-2),Arizona Diamondbacks
3,4,4,2013,Arizona Diamondbacks,NL West,162,81,81,0,0.500,...,685,695,"2,134,895",28.1,27.6,44,23,P.Goldschmidt (7.1),K.Gibson (81-81),Arizona Diamondbacks
4,5,5,2012,Arizona Diamondbacks,NL West,162,81,81,0,0.500,...,734,688,"2,177,617",28.3,27.4,48,23,A.Hill (5.0),K.Gibson (81-81),Arizona Diamondbacks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,2590,44,1973,Montreal Expos,NL East,162,79,83,0,0.488,...,668,702,"1,246,863",27.7,26.2,41,15,K.Singleton (5.7),G.Mauch (79-83),Washington Nationals
2590,2591,45,1972,Montreal Expos,NL East,156,70,86,0,0.449,...,513,609,"1,142,145",26.9,25.9,30,12,M.Marshall (4.0),G.Mauch (70-86),Washington Nationals
2591,2592,46,1971,Montreal Expos,NL East,162,71,90,1,0.441,...,622,729,"1,290,963",28.3,27.0,33,12,R.Staub (5.9),G.Mauch (71-90),Washington Nationals
2592,2593,47,1970,Montreal Expos,NL East,162,73,89,0,0.451,...,687,807,"1,424,683",27.7,26.7,40,17,R.Staub (6.3),G.Mauch (73-89),Washington Nationals


### First Metric for Correlation Analysis: Top Player WAR

In [5]:
# Split the `Top_Player` column to extract WAR
df[['Top_Player_Name', 'Top_Player_WAR']] = df['Top.Player'].str.extract(r'([A-Za-z\s.]+)\s\(([\d.]+)\)')
df['Top_Player_WAR'] = df['Top_Player_WAR'].astype(float)
df

,Unnamed: 0,Rk,Year,Tm,Lg,G,W,L,Ties,W.L.,...,Attendance,BatAge,PAge,X.Bat,X.P,Top.Player,Managers,current,Top_Player_Name,Top_Player_WAR
0,1,1,2016,Arizona Diamondbacks,NL West,162,69,93,0,0.426,...,"2,036,216",26.7,26.4,50,29,J.Segura (5.7),C.Hale (69-93),Arizona Diamondbacks,J.Segura,5.7
1,2,2,2015,Arizona Diamondbacks,NL West,162,79,83,0,0.488,...,"2,080,145",26.6,27.1,50,27,P.Goldschmidt (8.8),C.Hale (79-83),Arizona Diamondbacks,P.Goldschmidt,8.8
2,3,3,2014,Arizona Diamondbacks,NL West,162,64,98,0,0.395,...,"2,073,730",27.6,28.0,52,25,P.Goldschmidt (4.5),K.Gibson (63-96) and A.Trammell (1-2),Arizona Diamondbacks,P.Goldschmidt,4.5
3,4,4,2013,Arizona Diamondbacks,NL West,162,81,81,0,0.500,...,"2,134,895",28.1,27.6,44,23,P.Goldschmidt (7.1),K.Gibson (81-81),Arizona Diamondbacks,P.Goldschmidt,7.1
4,5,5,2012,Arizona Diamondbacks,NL West,162,81,81,0,0.500,...,"2,177,617",28.3,27.4,48,23,A.Hill (5.0),K.Gibson (81-81),Arizona Diamondbacks,A.Hill,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,2590,44,1973,Montreal Expos,NL East,162,79,83,0,0.488,...,"1,246,863",27.7,26.2,41,15,K.Singleton (5.7),G.Mauch (79-83),Washington Nationals,K.Singleton,5.7
2590,2591,45,1972,Montreal Expos,NL East,156,70,86,0,0.449,...,"1,142,145",26.9,25.9,30,12,M.Marshall (4.0),G.Mauch (70-86),Washington Nationals,M.Marshall,4.0
2591,2592,46,1971,Montreal Expos,NL East,162,71,90,1,0.441,...,"1,290,963",28.3,27.0,33,12,R.Staub (5.9),G.Mauch (71-90),Washington Nationals,R.Staub,5.9
2592,2593,47,1970,Montreal Expos,NL East,162,73,89,0,0.451,...,"1,424,683",27.7,26.7,40,17,R.Staub (6.3),G.Mauch (73-89),Washington Nationals,R.Staub,6.3


#### Utilize Helper Function calculate_playoff_score from Manager Predictive Model

In [21]:
# Use the existing function to calculate playoff scores
def calculate_playoff_score(playoff_result):
    if isinstance(playoff_result, str):
        if 'Lost ALWC' in playoff_result or 'Lost NLWC' in playoff_result:
            return 1
        elif 'Lost LDS' in playoff_result:
            return 2
        elif 'Lost ALCS' in playoff_result or 'Lost NLCS' in playoff_result:
            return 3
        elif 'Lost WS' in playoff_result or 'Tied in WS' in playoff_result:
            return 4
        elif 'Won WS' in playoff_result:
            return 5
    return 0

df['Playoff_Score'] = df['Playoffs'].apply(calculate_playoff_score)

#### Create a new DataFrame to store all top players along with metrics for future analysis

In [11]:
unique_top_players = df['Top_Player_Name'].unique()

player_df = pd.DataFrame(unique_top_players, columns=['Player_Name'])

player_df.reset_index(drop=True, inplace=True)

print(player_df)

        Player_Name
0          J.Segura
1     P.Goldschmidt
2            A.Hill
3           J.Upton
4           C.Young
...             ...
1178       A.Dawson
1179       S.Rogers
1180    M.Jorgensen
1181     M.Marshall
1182        R.Staub

[1183 rows x 1 columns]


### Getting Average WAR for each Player for First Metric in player_df

In [13]:
player_war_stats = df.groupby('Top_Player_Name')['Top_Player_WAR'].agg(['sum', 'mean']).reset_index()

# Rename columns for clarity
player_war_stats.columns = ['Player_Name', 'Total_WAR', 'Average_WAR']

# Merge the aggregated stats with player_df
player_df = pd.merge(player_df, player_war_stats, on='Player_Name', how='left')

print(player_df)

        Player_Name  Total_WAR  Average_WAR
0          J.Segura        5.7     5.700000
1     P.Goldschmidt       20.4     6.800000
2            A.Hill        5.0     5.000000
3           J.Upton       10.5     5.250000
4           C.Young       87.6     8.760000
...             ...        ...          ...
1178       A.Dawson       14.2     7.100000
1179       S.Rogers        9.5     4.750000
1180    M.Jorgensen        4.2     4.200000
1181     M.Marshall        4.0     4.000000
1182        R.Staub       18.4     6.133333

[1183 rows x 3 columns]


### Second Metric for Correlation Analysis: Top Player Win / Loss Averages

In [17]:
player_WL_stats = df.groupby('Top_Player_Name')['W.L.'].agg('mean').reset_index()

player_WL_stats.columns = ['Player_Name', 'Average_W.L.']

player_df = pd.merge(player_df, player_WL_stats, on='Player_Name', how='left')

print(player_df)

        Player_Name  Total_WAR  Average_WAR  Average_W.L.
0          J.Segura        5.7     5.700000      0.426000
1     P.Goldschmidt       20.4     6.800000      0.461000
2            A.Hill        5.0     5.000000      0.500000
3           J.Upton       10.5     5.250000      0.518500
4           C.Young       87.6     8.760000      0.523600
...             ...        ...          ...           ...
1178       A.Dawson       14.2     7.100000      0.556000
1179       S.Rogers        9.5     4.750000      0.401500
1180    M.Jorgensen        4.2     4.200000      0.491000
1181     M.Marshall        4.0     4.000000      0.449000
1182        R.Staub       18.4     6.133333      0.404333

[1183 rows x 4 columns]


### Third Metric for Correlation Analysis: Average Playoff Score

First, get total playoff score for each player

In [27]:
# Aggregate the Playoff Scores for Each Player
playoff_scores = df.groupby('Top_Player_Name')['Playoff_Score'].sum().reset_index()

# Rename the column for clarity
playoff_scores.rename(columns={'Top_Player_Name': 'Player_Name', 'Playoff_Score': 'Total_Playoff_Score'}, inplace=True)

# Merge the aggregated playoff scores with player_df
player_df = pd.merge(player_df, playoff_scores, on='Player_Name', how='left')

# Print the updated player_df with playoff scores
print(player_df.head(20))

      Player_Name  Total_WAR  Average_WAR  Average_W.L.  Total_Playoff_Score
0        J.Segura        5.7     5.700000      0.426000                    0
1   P.Goldschmidt       20.4     6.800000      0.461000                    0
2          A.Hill        5.0     5.000000      0.500000                    0
3         J.Upton       10.5     5.250000      0.518500                    2
4         C.Young       87.6     8.760000      0.523600                    5
5         D.Haren       13.7     6.850000      0.469000                    0
6          B.Webb       18.8     6.266667      0.514667                    3
7      C.Counsell        5.5     5.500000      0.475000                    0
8       R.Johnson       54.0     9.000000      0.529167                   12
9          O.Daal        4.1     4.100000      0.401000                    0
10      F.Freeman        6.5     6.500000      0.422000                    0
11      A.Simmons       40.9     5.842857      0.523714                    7

Now aggregate average Playoff Scores

In [29]:
# Aggregate the average Playoff Scores for Each Player
average_playoff_scores = df.groupby('Top_Player_Name')['Playoff_Score'].mean().reset_index()

# Rename the column for clarity
average_playoff_scores.rename(columns={'Top_Player_Name': 'Player_Name', 'Playoff_Score': 'Average_Playoff_Score'}, inplace=True)

# Merge the aggregated average playoff scores with player_df
player_df = pd.merge(player_df, average_playoff_scores, on='Player_Name', how='left')

# Print the updated player_df with average playoff scores
print(player_df.head(20))

      Player_Name  Total_WAR  Average_WAR  Average_W.L.  Total_Playoff_Score  \
0        J.Segura        5.7     5.700000      0.426000                    0   
1   P.Goldschmidt       20.4     6.800000      0.461000                    0   
2          A.Hill        5.0     5.000000      0.500000                    0   
3         J.Upton       10.5     5.250000      0.518500                    2   
4         C.Young       87.6     8.760000      0.523600                    5   
5         D.Haren       13.7     6.850000      0.469000                    0   
6          B.Webb       18.8     6.266667      0.514667                    3   
7      C.Counsell        5.5     5.500000      0.475000                    0   
8       R.Johnson       54.0     9.000000      0.529167                   12   
9          O.Daal        4.1     4.100000      0.401000                    0   
10      F.Freeman        6.5     6.500000      0.422000                    0   
11      A.Simmons       40.9     5.84285

#### Before getting final metric, create a new column to store boolean for each row based on if they won the World Series

In [35]:
# Create the World_Series_Win column
df['World_Series_Win'] = df['Playoff_Score'].apply(lambda x: 1 if x == 5 else 0)


### Forth and Final Metric for Correlation Analysis: Average World Series Winner

In [37]:
# Compute the total number of World Series wins for each player
total_world_series_wins = df.groupby('Top_Player_Name')['World_Series_Win'].sum().reset_index()

# Compute the number of seasons each player played
total_seasons = df.groupby('Top_Player_Name').size().reset_index(name='Total_Seasons')

# Merge total World Series wins and total seasons
player_world_series = pd.merge(total_world_series_wins, total_seasons, on='Top_Player_Name')

# Compute the average World Series championships per season
player_world_series['Average_World_Series'] = player_world_series['World_Series_Win'] / player_world_series['Total_Seasons']

# Rename columns for clarity
player_world_series.rename(columns={'Top_Player_Name': 'Player_Name', 'Average_World_Series': 'Average_World_Series_Championships'}, inplace=True)

# Merge the aggregated average World Series championships with player_df
player_df = pd.merge(player_df, player_world_series[['Player_Name', 'Average_World_Series_Championships']], on='Player_Name', how='left')

# Print the updated player_df with average World Series championships
print(player_df.head(20))

      Player_Name  Total_WAR  Average_WAR  Average_W.L.  Total_Playoff_Score  \
0        J.Segura        5.7     5.700000      0.426000                    0   
1   P.Goldschmidt       20.4     6.800000      0.461000                    0   
2          A.Hill        5.0     5.000000      0.500000                    0   
3         J.Upton       10.5     5.250000      0.518500                    2   
4         C.Young       87.6     8.760000      0.523600                    5   
5         D.Haren       13.7     6.850000      0.469000                    0   
6          B.Webb       18.8     6.266667      0.514667                    3   
7      C.Counsell        5.5     5.500000      0.475000                    0   
8       R.Johnson       54.0     9.000000      0.529167                   12   
9          O.Daal        4.1     4.100000      0.401000                    0   
10      F.Freeman        6.5     6.500000      0.422000                    0   
11      A.Simmons       40.9     5.84285